## Using Komo for IK

In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



In [2]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

q_0 = C.getJointState()

In [3]:
obj = C.addFrame("object")
obj.setPosition([.3,0,.73])
#obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.ssBox, [.2,.05,0.04,0.01])
obj.setColor([1,1,0])
obj.setContact(1)


In [4]:
mk_obj = C.addFrame("mk_obj","object")
mk_obj.setShape(ry.ST.marker,[0.2])


In [5]:
mk_gripper = C.addFrame("mk_gripper","R_gripper")
mk_gripper.setShape(ry.ST.marker,[0.1])

mk_real = C.addFrame("mk_real_gripper")
mk_real.setShape(ry.ST.marker,[.1])


In [6]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex02_4.g")
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

In [7]:
#-- the following is the simulation loop
tau = .01

for t in range(200):
    time.sleep(0.01)
    q = S.get_q()
    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    C.setFrameState(RealWorld.getFrameState("obj1"),["object"])
    #send velocity controls to the simulation
    S.step(np.zeros_like(q), tau, ry.ControlMode.velocity)

In [8]:


# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,30, 5., True)
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripper", "object"], ry.OT.sos, [1e2])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.addObjective([.7,1.], ry.FS.position, ["R_gripper"], ry.OT.eq, [1e2],[-.0,-.1,-.0], order=2)
komo.addObjective([], ry.FS.distance, ["R_gripper", "object"], ry.OT.ineq, [1e2],[-.0])
komo.addObjective([1.], 
                ry.FS.scalarProductXX, 
                ["object","R_gripper"], 
                ry.OT.eq, 
                [1e2],
                [0]);
komo.addObjective([1.], 
                ry.FS.scalarProductZZ, 
                ["object","R_gripper"], 
                ry.OT.eq, 
                [1e2],
                [1]);
print(komo.getT(),komo.getPathFrames().shape)

komo.optimize()
#komo.getReport()
#V = komo.view_play(False, 1)



30 (30, 101, 7)


In [13]:


for frame,tau in zip(komo.getPathFrames(),komo.getPathTau()):
    time.sleep(tau)
    print(RealWorld.getFrameState("R_gripper")[:])
    mk_real.setPosition(RealWorld.getFrameState("R_gripper")[:3])
    mk_real.setQuaternion(RealWorld.getFrameState("R_gripper")[3:])
    #some good old fashioned IK
    C.setFrameState(frame)

    [y,J] = C.evalFeature(ry.FS.poseDiff, ["R_gripper", "mk_real_gripper"])

    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

[-0.17737767 -0.38415171  0.87845634 -0.08155722 -0.50500443  0.32426706
 -0.79571969]


AttributeError: 'libry.Config' object has no attribute 'getPosition'

In [14]:
C.getFrameNames()

['world',
 'table',
 'L_panda_link0',
 'L_panda_link0_1',
 'L_panda_link0>panda_joint1',
 'L_panda_joint1',
 'L_panda_link1_1',
 'L_panda_link1>panda_joint2',
 'L_panda_joint2',
 'L_panda_link2_1',
 'L_panda_link2>panda_joint3',
 'L_panda_joint3',
 'L_panda_link3_1',
 'L_panda_link3>panda_joint4',
 'L_panda_joint4',
 'L_panda_link4_1',
 'L_panda_link4>panda_joint5',
 'L_panda_joint5',
 'L_panda_link5_1',
 'L_panda_link5>panda_joint6',
 'L_panda_joint6',
 'L_panda_link6_1',
 'L_panda_link6>panda_joint7',
 'L_panda_joint7',
 'L_panda_link7_1',
 'L_panda_link7>panda_joint8',
 'L_panda_joint8',
 'L_panda_link8>panda_hand_joint',
 'L_panda_hand_joint',
 'L_panda_hand_1',
 'L_panda_hand>panda_finger_joint1',
 'L_panda_hand>panda_finger_joint2',
 'L_panda_finger_joint1',
 'L_panda_finger_joint2',
 'L_panda_leftfinger_1',
 'L_panda_rightfinger_1',
 'L_panda_coll0',
 'L_panda_coll0b',
 'L_panda_coll1',
 'L_panda_coll3',
 'L_panda_coll5',
 'L_panda_coll2',
 'L_panda_coll4',
 'L_panda_coll6',
 'L